<a href="https://colab.research.google.com/github/AlphaKermit-999/AI-Topics-learnings/blob/main/Advanced_AI/LLM_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
model_name = "google/flan-t5-small"

In [2]:
import pandas as pd
df = pd.read_csv("/content/customer_feedback.csv", sep=", ")

/tmp/ipython-input-3618132263.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("/content/customer_feedback.csv", sep=", ")


In [3]:
df.head()

,"""Text",Sentiment,Source,Date/Time,User ID,Location,"Confidence Score"""
0,"""""""I love this product!""""",Positive,Twitter,2023-06-15 09:23:14,@user123,New York,"0.85"""
1,"""""""The service was terrible.""""",Negative,Yelp Reviews,2023-06-15 11:45:32,user456,Los Angeles,"0.65"""
2,"""""""This movie is amazing!""""",Positive,IMDb,2023-06-15 14:10:22,moviefan789,London,"0.92"""
3,"""""""I'm so disappointed with their customer sup...",Negative,Online Forum,2023-06-15 17:35:11,forumuser1,Toronto,"0.78"""
4,"""""""Just had the best meal of my life!""""",Positive,TripAdvisor,2023-06-16 08:50:59,foodie22,Paris,"0.88"""


In [13]:
df["Sentiment"].values

array(['Positive', 'Negative', 'Positive', 'Negative', 'Positive',
       'Negative', 'Positive', 'Positive', 'Positive', 'Negative',
       'Positive', 'Negative', 'Positive', 'Negative', 'Positive',
       'Negative', 'Positive', 'Negative', 'Positive', 'Negative',
       'Positive', 'Positive', 'Negative', 'Negative', 'Negative',
       'Positive', 'Positive', 'Positive', 'Negative', 'Positive',
       'Negative', 'Positive', 'Positive', 'Positive', 'Negative',
       'Positive', 'Positive', 'Negative', 'Positive', 'Negative',
       'Positive', 'Negative', 'Positive', 'Negative', 'Positive',
       'Positive', 'Positive', 'Negative', 'Positive', 'Negative',
       'Negative', 'Positive', 'Positive', 'Negative', 'Positive',
       'Negative', 'Negative', 'Negative', 'Positive', 'Negative',
       'Positive', 'Negative', 'Positive', 'Negative', 'Positive',
       'Negative', 'Positive', 'Negative', 'Positive', 'Positive',
       'Positive', 'Negative', 'Positive', 'Positive', 'Negati

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [17]:
tokeniser = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [18]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [19]:
model = model.to(device)

In [35]:
def sentiment(data):
  prompt = "classify the sentiment of the following customer feedback as Positive or Negative :\n Feedback: {data} \n Sentiment"
  inputs = tokeniser(prompt, return_tensors="pt").to(device)
  output = model.generate(**inputs,
                          top_p=0.9,
                          top_k=50,
                          max_new_tokens=100,
                          min_new_tokens=10,
                          temperature=0.6,
                          do_sample=True)
  return tokeniser.decode(output[0], skip_special_tokens=True)


In [36]:
fb = df['"Text'].tolist()
pred = [sentiment(i) for i in fb]

In [ ]:
df["predicted"] = df['"Text'].apply(sentiment)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(df["predicted"], df["Sentiment"])

In [ ]:
print(classification_report(df["predicted"], df["Sentiment"]))